# 🚀 Fine-tuning de LLM para Auditoría de Calidad

Este cuaderno permite entrenar un modelo de lenguaje (LLM) especializado en auditoría de llamadas de servicio al cliente, utilizando los resultados generados por el pipeline anterior.

### Características:
- **Motor**: [Unsloth](https://github.com/unslothai/unsloth) (2x más rápido, 70% menos memoria).
- **Método**: QLoRA (Fine-tuning eficiente en 4-bit).
- **Modelos soportados**: Llama 3.1, Qwen 2.5, Mistral.
- **Salida**: Formato GGUF para usar en **LM Studio**.

## 0. 📊 Generación de Data de Entrenamiento (Punto 0)
En esta sección generamos el dataset para el fine-tuning combinando:
1. **Reglas de Negocio**: Extraídas de `indicaciones_gestion_requerimiento.json`.
2. **Análisis Reales**: Resultados previos de la carpeta `greeting_analysis`.
3. **Escenarios Sintéticos**: Variaciones generadas para cubrir casos de cumplimiento e incumplimiento.

In [1]:
import json
import os
import random
from pathlib import Path

CRITERIA_FILE = Path(r"./prompt/indicaciones_gestion_requerimiento.json")
ANALYSIS_DIR = Path(r"./output/greeting_analysis")
OUTPUT_DIR = Path(r"./data/pomptsft")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
DATASET_FILE = OUTPUT_DIR / "dataset_audit.jsonl"

def generate_all_rules_data(n_samples=2500):
    """Generador masivo que cubre R1 hasta R9 con todas sus variantes."""
    
    names = ["Juan Pérez", "María García", "Carlos Ruiz", "Ana Torres", "Luis Vega", "Elena Sol"]
    dnis = ["11223344", "55667788", "99001122", "33445566", "77889900"]
    dates = ["01/01/1980", "15/05/1992", "20/12/1975", "10/10/1988"]
    places = ["Lima", "Cusco", "Arequipa", "Trujillo", "Piura"]
    amounts = ["45.50", "89.90", "120.00", "35.00", "150.25"]
    
    dataset = []
    print(f"🚀 Generando {n_samples} ejemplos de entrenamiento con cobertura R1-R9...")
    
    for _ in range(n_samples):
        dialog = []
        analysis = {}
        intent = random.choice(["BAJA", "CONSULTA", "RECLAMO", "BAJA_PREVIA"])
        name, dni, date, place, amt = [random.choice(lst) for lst in [names, dnis, dates, places, amounts]]
        
        # --- R1 / R1A: Validación ---
        if intent == "BAJA":
            is_compliant = random.choice([True, False])
            is_interrupted = random.random() < 0.1 # 10% de probabilidad de corte
            
            if is_interrupted:
                dialog.append(f"Cliente: Quiero la baja.\nAsesor: Por favor valide: nombre, dni, fecha, lugar y monto.\nCliente: {name}, {dni}... espere...\n(Llamada cortada por cliente)")
                analysis["R1_validacion_datos"] = {"cumple": True, "razon": "Asesor aplicó protocolo de baja antes del corte.", "score": 10}
                analysis["R9_falta_informacion"] = {"cumple": "NO APLICA", "razon": "Corte por cliente impide terminar.", "score": 0}
            elif is_compliant:
                dialog.append(f"Cliente: Quiero la baja.\nAsesor: Necesito su Nombre, DNI, Fecha, Lugar y Monto.\nCliente: {name}, {dni}, {date}, {place}, {amt} soles.\nAsesor: Validado correctamente.")
                analysis["R1_validacion_datos"] = {"cumple": True, "razon": "Validación completa realizada.", "score": 10}
            else:
                dialog.append(f"Cliente: Quiero dar de baja mi línea.\nAsesor: ¿DNI y nombre?\nCliente: {dni}, {name}.\nAsesor: Listo, procederemos.")
                analysis["R1_validacion_datos"] = {"cumple": False, "razon": "Omitió campos obligatorios (fecha, lugar, monto) para baja.", "score": 0}
        else:
            dialog.append(f"Cliente: Tengo una duda.\nAsesor: Nombre y DNI.\nCliente: {name}, {dni}.\nAsesor: ¿En qué le ayudo?")
            analysis["R1_validacion_datos"] = {"cumple": True, "razon": "Identificación básica suficiente para gestión no sensible.", "score": 10}

        # --- R2: Empatía (Randomly add empathy issues) ---
        r2_compliant = random.random() > 0.15
        if not r2_compliant:
            dialog.append("Asesor: (Tono molesto) Apúrese que tengo más llamadas.\nCliente: No me hable así.")
            analysis["R2_empatia_claridad"] = {"cumple": False, "razon": "Asesor muestra falta de cortesía y tono confrontativo.", "score": 0}
        else:
            analysis["R2_empatia_claridad"] = {"cumple": True, "razon": "Mantiene comunicación clara y respetuosa.", "score": 10}

        # --- R3 / R4 / R8: Retenciones y Rebatimientos ---
        if intent == "BAJA":
            n_offers = random.randint(1, 5)
            for i in range(min(n_offers, 3)):
                dialog.append(f"Asesor: ¿Le interesa la oferta {i+1}?\nCliente: No gracias.")
            if n_offers > 3:
                dialog.append(f"Asesor: Insisto con la oferta {n_offers}.\nCliente: Ya dije que no.")
                analysis["R3_ofertas_adecuadas"] = {"cumple": False, "razon": "Superó el límite de 3 ofertas.", "score": 0}
                analysis["R4_respeto_decision"] = {"cumple": False, "razon": "No respetó la decisión tras 3 intentos.", "score": 0}
                analysis["R8_regla_rebatimientos"] = {"cumple": False, "razon": "Superó límite de rebatimientos permitidos.", "score": 0}
            else:
                analysis["R3_ofertas_adecuadas"] = {"cumple": True, "razon": "Dentro del límite de 3 ofertas.", "score": 10}
                analysis["R4_respeto_decision"] = {"cumple": True, "razon": "Respeta decisión tras ofertas permitidas.", "score": 10}
                analysis["R8_regla_rebatimientos"] = {"cumple": True, "razon": "Rebatimientos realizados según política.", "score": 10}
        else:
            # Si no es baja, no debe ofrecer retenciones
            should_offer = random.choice([True, False]) if intent == "CONSULTA" else False
            if should_offer:
                 dialog.append("Asesor: Le ofrezco un descuento.\nCliente: Pero solo pregunté mi saldo.")
                 # En consulta/facilidades no penaliza si ofrece? La regla dice: 
                 # "Si no manifiesta deseo de baja... el asesor NO debe ofrecer retenciones... se considera CUMPLE al mantener el contexto"
                 # En el prompt original de la regla dice que "se considera CUMPLE al mantener la conversación dentro del contexto correcto".
                 # Interpretación: Si ofrece cuando NO es baja, está fuera de contexto? 
                 # Corrijo: En consulta no debe ofrecer. Si ofrece, es NO CUMPLE en R3.
                 analysis["R3_ofertas_adecuadas"] = {"cumple": False, "razon": "Ofreció retenciones en una gestión que no era de baja.", "score": 0}
            else:
                 analysis["R3_ofertas_adecuadas"] = {"cumple": True, "razon": "No realizó ofertas fuera de contexto.", "score": 10}

        # --- R6A: Baja Previa ---
        if intent == "BAJA_PREVIA":
            r6a_compliant = random.choice([True, False])
            if r6a_compliant:
                dialog.append(f"Asesor: El código de su baja anterior es {random.randint(111,999)}.")
                analysis["R6A_consulta_baja_previa"] = {"cumple": True, "razon": "Brinda información de baja previa correctamente.", "score": 10}
            else:
                dialog.append("Asesor: No veo nada de bajas anteriores en mi pantalla.")
                analysis["R6A_consulta_baja_previa"] = {"cumple": False, "razon": "No asistió con información de gestión anterior.", "score": 0}

        # --- R7: Tiempo de Espera ---
        wait_time = random.randint(1, 15)
        if wait_time > 5 and intent == "BAJA":
            dialog.append(f"(Espera silenciosa de {wait_time} minutos sin motivo)")
            analysis["R7_tiempo_espera_justificado"] = {"cumple": False, "razon": f"Espera excesiva de {wait_time} min para dilatar baja.", "score": 0}
        else:
            analysis["R7_tiempo_espera_justificado"] = {"cumple": True, "razon": "Tiempo en espera razonable.", "score": 10}

        # --- R5 / R6: Cierre y Resolución ---
        cuts_at_end = random.random() < 0.05
        if cuts_at_end:
            dialog.append("(Cliente se desconecta antes de formalizar)")
            analysis["R5_formalizacion_cierre"] = {"cumple": "NO APLICA", "razon": "Corte imprevisto por cliente.", "score": 0}
            analysis["R6_resolver_consulta_asociada"] = {"cumple": "NO APLICA", "razon": "Corte impide resolución final.", "score": 0}
        else:
            r5_compliant = random.choice([True, False])
            if r5_compliant:
                dialog.append(f"Asesor: Gestión realizada. Código: ID-{random.randint(1,100)}, plazo 24h.")
                analysis["R5_formalizacion_cierre"] = {"cumple": True, "razon": "Formaliza con código y plazos.", "score": 10}
                analysis["R6_resolver_consulta_asociada"] = {"cumple": True, "razon": "Resuelve la solicitud principal.", "score": 10}
            else:
                dialog.append("Asesor: Listo, chau.")
                analysis["R5_formalizacion_cierre"] = {"cumple": False, "razon": "No entregó código ni explicó plazos.", "score": 0}
                analysis["R6_resolver_consulta_asociada"] = {"cumple": False, "razon": "No brindó información final necesaria.", "score": 0}

        dataset.append({
            "instruction": "Genera un reporte de auditoría completo basado en esta transcripción, evaluando las reglas R1 a R9.",
            "input": "\n".join(dialog),
            "output": json.dumps({"rule_analysis": analysis}, ensure_ascii=False)
        })
    
    # Integración de data real
    if ANALYSIS_DIR.exists():
        real_files = list(ANALYSIS_DIR.glob("*.json"))
        for f_path in real_files:
            try:
                with open(f_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                if "transcription_text" in data and "rule_analysis" in data:
                    dataset.append({
                        "instruction": "Genera un reporte de auditoría completo basado en esta transcripción.",
                        "input": data["transcription_text"],
                        "output": json.dumps({"rule_analysis": data["rule_analysis"]}, ensure_ascii=False)
                    })
            except: continue

    random.shuffle(dataset)
    with open(DATASET_FILE, 'w', encoding='utf-8') as f:
        for entry in dataset:
            f.write(json.dumps(entry, ensure_ascii=False) + "\n")
    
    print(f"🎉 Dataset total construido con {len(dataset)} ejemplos (sintéticos + reales).")
    print(f"📁 Archivo guardado en: {DATASET_FILE}")

generate_all_rules_data(n_samples=3000)


🚀 Generando 3000 ejemplos de entrenamiento con cobertura R1-R9...
🎉 Dataset total construido con 3000 ejemplos (sintéticos + reales).
📁 Archivo guardado en: data\pomptsft\dataset_audit.jsonl


## 1. 🛠️ Configuración del Entorno
Instalación de dependencias optimizadas.

In [ ]:
!pip install --no-deps unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git
!pip install --no-deps "xformers<0.0.29" "trl<0.9.0" peft accelerate bitsandbytes
!pip install pandas datasets

## 2. 🧠 Carga del Modelo Base
Cargamos el modelo en 4 bits para ahorrar memoria.

In [4]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Sube a 4096 si tienes mucha VRAM
model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit" # O "unsloth/Qwen2.5-7B-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    load_in_4bit = True,
)

# Agregar adaptadores LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

ImportError: Unsloth: torch==2.9.1 requires torchvision>=0.24.0, but found torchvision==0.17.0+cu118. Please refer to https://pytorch.org/get-started/previous-versions/ for more information.

## 3. 📊 Preparación de Datos
Convertimos los JSON del pipeline anterior al formato **Alpaca**.

In [6]:
import json
import os
import pandas as pd
from pathlib import Path
from datasets import load_dataset
from transformers import AutoTokenizer


# 🔧 Configuración de rutas
DATA_DIR = Path("./data/pomptsft")
DATASET_FILE = DATA_DIR / "dataset_audit.jsonl"
ALPACA_TEMPLATE = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{} 

### Input:
{} 

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        # Formatear cada ejemplo con el template de Alpaca
        text = ALPACA_TEMPLATE.format(instruction, input_text, output) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }

def load_and_prepare_dataset():
    print(f"📂 Cargando dataset desde: {DATASET_FILE}")
    
    if not DATASET_FILE.exists():
        print("❌ ERROR: El archivo de dataset no existe. Ejecuta el Punto 0 primero.")
        return None
        
    # Cargar usando la librería datasets de HuggingFace
    dataset = load_dataset("json", data_files=str(DATASET_FILE), split="train")
    
    # Aplicar el formateo de Alpaca (mapeo para el entrenamiento)
    dataset = dataset.map(formatting_prompts_func, batched = True,)
    
    print(f"✅ Dataset cargado y formateado con {len(dataset)} ejemplos.")
    return dataset

# Ejecutar la carga
dataset = load_and_prepare_dataset()

# Mostrar un ejemplo del texto final que verá el LLM
if dataset:
    print("\n--- MUESTRA DEL FORMATO FINAL ---")
    print(dataset[0]["text"])


📂 Cargando dataset desde: data\pomptsft\dataset_audit.jsonl


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

NameError: name 'tokenizer' is not defined

## 4. 🔥 Entrenamiento
Ejecución del ciclo de fine-tuning.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset

dataset = load_dataset("json", data_files=OUTPUT_DATASET, split="train")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text", # O usar formato Alpaca directo
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Ajustar según tamaño de dataset
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

trainer.train()

## 5. 📦 Exportación a GGUF (LM Studio)
Guardar el modelo para usarlo localmente.

In [ ]:
# Guardar en formato GGUF (esto permite cargarlo en LM Studio)
model.save_pretrained_gguf("model_audit_q4", tokenizer, quantization_method = "q4_k_m")

print("✅ Modelo exportado correctamente a 'model_audit_q4'")
print("Busca el archivo .gguf para cargarlo en LM Studio.")